In [19]:
import os
import contextlib

import pandas as pd
import seaborn as sns
from datetime import date
import plotly.express as px
import plotly.graph_objects as go

In [20]:
def find_sheet_name(sheet_names):
    gdp_sheet_names = list(filter(lambda x: "gdp-ss" in x.lower(), sheet_names))

    if len(gdp_sheet_names) == 1:
        return gdp_sheet_names[0]

    gdp_sheet_names = list(
        filter(lambda x: "gdp" in x.lower() and "ss" in x.lower(), sheet_names)
    )

    if len(gdp_sheet_names) == 1:
        return gdp_sheet_names[0]

    gdp_sheet_names = list(filter(lambda x: "gdp" in x.lower(), sheet_names))

    if len(gdp_sheet_names) == 1:
        return gdp_sheet_names[0]

    if len(sheet_names) == 43:
        return "2"

    raise RuntimeError(sheet_names)

In [21]:
en_to_vn = {
    "TOTAL": "TỔNG SỐ",
    "Agriculture, Forestry and Fishing": "Nông, lâm nghiệp và thủy sản",
    "Agriculture": "Nông nghiệp",
    "Forestry": "Lâm nghiệp",
    "Fishery": "Thủy sản",
    "Industry and Construction": "Công nghiệp và xây dựng",
    "Industry": "Công nghiệp",
    "Mining and quarrying": "Khai khoáng",
    "Manufacturing": "Công nghiệp chế biến, chế tạo",
    "Production & supply of electricity, gas, hot water, steam and air-conditioning": "Sản xuất và phân phối điện, khí đốt, nước nóng, hơi nước và điều hòa không khí",
    "Water supply; sewerage, waste management and remediation activities": "Cung cấp nước; hoạt động quản lý và xử lý rác thải, nước thải",
    "Construction": "Xây dựng",
    "Services": "Dịch vụ",
    "Wholesales, retails, vehicles and motorcycles repairs": "Bán buôn và bán lẻ; sửa chữa ô tô, mô tô, xe máy và xe có động cơ khác",
    "Transportation and storage": "Vận tải, kho bãi",
    "Accommodation & catering services": "Dịch vụ lưu trú và ăn uống",
    "Information & communication": "Thông tin và truyền thông",
    "Finance, banking & insurance": "Hoạt động tài chính, ngân hàng và bảo hiểm",
    "Real estate business": "Hoạt động kinh doanh bất động sản",
    "Scientific and technical specialty": "Hoạt động chuyên môn, khoa học và công nghệ",
    "Administrative activity and supporting service": "Hoạt động hành chính và dịch vụ hỗ trợ",
    "Activities of Communist Party, socio-political organizations; State management and national defense; compulsory social security": "Hoạt động của Đảng Cộng sản, tổ chức chính trị- xã hội; quản lý Nhà nước, an ninh quốc phòng; đảm bảo xã hội bắt buộc",
    "Education and training": "Giáo dục và đào tạo",
    "Health care & social relief": "Y tế và hoạt động trợ giúp xã hội",
    "Art, entertainment and recreation": "Nghệ thuật, vui chơi và giải trí",
    "Other service activities": "Hoạt động dịch vụ khác",
    "Activities of households as employers; undifferentiated goods and services producing activities of households for own use": "Hoạt động làm thuê các công việc trong các hộ gia đình, sản xuất sản phẩm vật chất và dịch vụ tự tiêu dùng của hộ gia đình",
    "Products taxes less subsidies on production": "Thuế sản phẩm trừ trợ cấp sản phẩm",
}

In [22]:
def get_gdp_ss(xl, sheet_name, date):
    df = xl.parse(sheet_name)

    df[df.columns[0]] = df[df.columns[0]].fillna(df[df.columns[1]])

    if len(df.columns) == 8:
        df = df.drop(
            columns=[
                df.columns[1],
                df.columns[2],
                df.columns[4],
                # df.columns[5],
                df.columns[6],
                df.columns[7],
            ]
        )
    elif len(df.columns) == 7:
        df = df.drop(
            columns=[
                df.columns[1],
                df.columns[2],
                df.columns[3],
                df.columns[4],
                # df.columns[6],
            ]
        )
    elif len(df.columns) == 5 and int(date.split("-")[1]) == 12:
        df = df.drop(columns=[df.columns[1], df.columns[2]])
    elif len(df.columns) == 6 and int(date.split("-")[1]) == 12:
        df = df.drop(columns=[df.columns[1], df.columns[2], df.columns[5]])
    else:
        raise RuntimeError(df.columns)

    assert len(df.columns) == 3
    df.columns = ["Ngành", "Tổng số", "YOY"]

    df[df.columns[0]] = (
        df[df.columns[0]]
        .replace(r"\s+", " ", regex=True)
        .replace(r"\n", " ", regex=True)
        .str.strip()
    )

    df = df.drop(
        list(
            range(
                df[df.columns[0]][
                    (df[df.columns[0]] == "TỔNG SỐ")
                    | (df[df.columns[0]] == "TOTAL")
                    | (df[df.columns[0]] == "TỔNG SẢN PHẨM TRONG NƯỚC")
                ].index[0]
            )
        )
    )

    if len(df["Ngành"][df["Ngành"] == "TOTAL"].index) > 0:
        df["Ngành"] = df["Ngành"].map(en_to_vn)

    df = df[pd.to_numeric(df[df.columns[1]], errors="coerce").notnull()]

    if int(date.split("-")[0]) < 2024:
        df["YOY"] = df["YOY"] - 100

    df["Date"] = date
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=False)

    df["Tổng số"] = df["Tổng số"].astype(float)

    total = df.loc[df["Tổng số"].idxmax(), df.columns[1]]
    df["Percent"] = df["Tổng số"] / total

    return df


xl = pd.ExcelFile("../datas/2024-03-29-02-Bieu-3.2024-ngay-28.3-1.xlsx")

sheet_name = find_sheet_name(xl.sheet_names)

get_gdp_ss(xl, sheet_name, "2024-03-29")

,Ngành,Tổng số,YOY,Date,Percent
6,TỔNG SỐ,1.396660e+06,5.657544,2024-03-29,1.000000
7,"Nông, lâm nghiệp và thủy sản",1.456001e+05,2.979307,2024-03-29,0.104249
8,Nông nghiệp,1.122149e+05,2.805032,2024-03-29,0.080345
9,Lâm nghiệp,5.908701e+03,4.083476,2024-03-29,0.004231
10,Thủy sản,2.747649e+04,3.459557,2024-03-29,0.019673
11,Công nghiệp và xây dựng,4.882085e+05,6.281638,2024-03-29,0.349554
12,Công nghiệp,4.127400e+05,6.181939,2024-03-29,0.295519
13,Khai khoáng,3.818409e+04,-5.839812,2024-03-29,0.027340
14,"Công nghiệp chế biến, chế tạo",3.168565e+05,6.979778,2024-03-29,0.226867
15,"Sản xuất và phân phối điện, khí đốt, nước nóng...",5.025100e+04,11.968187,2024-03-29,0.035979


In [23]:
import os
import contextlib

dfs_gdp_ss_origin = []
for file_name in sorted(
    os.listdir("../datas"),
    key=lambda filename: f"{filename.split('-')[0]}-{filename.split('-')[1]}-{filename.split('-')[2]}",
    reverse=True,
):
    if file_name.startswith(".~"):
        continue

    try:
        file_path = os.path.join("../datas", file_name)

        year = int(file_name.split("-")[0])
        month = int(file_name.split("-")[1])
        day = int(file_name.split("-")[2])

        if year < 2014:
            continue

        if month not in [3, 6, 9, 12]:
            continue

        xl = pd.ExcelFile(file_path)

        df_gdp = get_gdp_ss(
            xl, find_sheet_name(xl.sheet_names), f"{year}-{month}-{day}"
        )

        dfs_gdp_ss_origin.append(df_gdp)
    except Exception as e:
        print(file_name, str(e))

In [24]:
dfs_gdp_ss = pd.concat(dfs_gdp_ss_origin, ignore_index=True, axis=0)
dfs_gdp_ss["Date"] = pd.to_datetime(dfs_gdp_ss["Date"], dayfirst=False)
dfs_gdp_ss

,Ngành,Tổng số,YOY,Date,Percent
0,TỔNG SỐ,1.396660e+06,5.657544,2024-03-29,1.000000
1,"Nông, lâm nghiệp và thủy sản",1.456001e+05,2.979307,2024-03-29,0.104249
2,Nông nghiệp,1.122149e+05,2.805032,2024-03-29,0.080345
3,Lâm nghiệp,5.908701e+03,4.083476,2024-03-29,0.004231
4,Thủy sản,2.747649e+04,3.459557,2024-03-29,0.019673
...,...,...,...,...,...
1141,Giáo dục và đào tạo,1.430664e+04,7.82,2014-03-28,0.028404
1142,Y tế và hoạt động trợ giúp xã hội,6.434305e+03,7.80182,2014-03-28,0.012775
1143,"Nghệ thuật, vui chơi và giải trí",3.831353e+03,9.12,2014-03-28,0.007607
1144,Hoạt động dịch vụ khác,9.223516e+03,7.44,2014-03-28,0.018312


In [25]:
convert_nganh = {
    "TỔNG SỐ": "TỔNG SỐ",
    "Nông, lâm nghiệp và thủy sản": "Nông, lâm nghiệp và thủy sản",
    "Nông nghiệp": "Nông nghiệp",
    "Lâm nghiệp": "Lâm nghiệp",
    "Thủy sản": "Thủy sản",
    "Công nghiệp và xây dựng": "Công nghiệp và xây dựng",
    "Công nghiệp": "Công nghiệp",
    "Khai khoáng": "Khai khoáng",
    "Công nghiệp chế biến, chế tạo": "Công nghiệp chế biến, chế tạo",
    "Sản xuất và phân phối điện, khí đốt, nước nóng, hơi nước và điều hòa không khí": "Sản xuất và phân phối điện, khí đốt, nước nóng, hơi nước và điều hòa không khí",
    "Cung cấp nước; hoạt động quản lý và xử lý rác thải, nước thải": "Cung cấp nước; hoạt động quản lý và xử lý rác thải, nước thải",
    "Xây dựng": "Xây dựng",
    "Dịch vụ": "Dịch vụ",
    "Bán buôn và bán lẻ; sửa chữa ô tô, mô tô, xe máy và xe có động cơ khác": "Bán buôn và bán lẻ; sửa chữa ô tô, mô tô, xe máy và xe có động cơ khác",
    "Vận tải, kho bãi": "Vận tải, kho bãi",
    "Dịch vụ lưu trú và ăn uống": "Dịch vụ lưu trú và ăn uống",
    "Thông tin và truyền thông": "Thông tin và truyền thông",
    "Hoạt động tài chính, ngân hàng và bảo hiểm": "Hoạt động tài chính, ngân hàng và bảo hiểm",
    "Hoạt động kinh doanh bất động sản": "Hoạt động kinh doanh bất động sản",
    "Hoạt động chuyên môn, khoa học và công nghệ": "Hoạt động chuyên môn, khoa học và công nghệ",
    "Hoạt động hành chính và dịch vụ hỗ trợ": "Hoạt động hành chính và dịch vụ hỗ trợ",
    "Hoạt động của Đảng Cộng sản, tổ chức chính trị-xã hội; quản lý Nhà nước, an ninh quốc phòng; đảm bảo xã hội bắt buộc": "Hoạt động của Đảng Cộng sản, tổ chức chính trị-xã hội; quản lý Nhà nước, an ninh quốc phòng; đảm bảo xã hội bắt buộc",
    "Giáo dục và đào tạo": "Giáo dục và đào tạo",
    "Y tế và hoạt động trợ giúp xã hội": "Y tế và hoạt động trợ giúp xã hội",
    "Nghệ thuật, vui chơi và giải trí": "Nghệ thuật, vui chơi và giải trí",
    "Hoạt động dịch vụ khác": "Hoạt động dịch vụ khác",
    "Hoạt động làm thuê các công việc trong các hộ gia đình, sản xuất sản phẩm vật chất và dịch vụ tự tiêu dùng của hộ gia đình": "Hoạt động làm thuê các công việc trong các hộ gia đình, sản xuất sản phẩm vật chất và dịch vụ tự tiêu dùng của hộ gia đình",
    "Thuế sản phẩm trừ trợ cấp sản phẩm": "Thuế sản phẩm trừ trợ cấp sản phẩm",
    "Hoạt động của Đảng Cộng sản, tổ chức chính trị- xã hội; quản lý Nhà nước, an ninh quốc phòng; đảm bảo xã hội bắt buộc": "Hoạt động của Đảng Cộng sản, tổ chức chính trị-xã hội; quản lý Nhà nước, an ninh quốc phòng; đảm bảo xã hội bắt buộc",
    "TỔNG SẢN PHẨM TRONG NƯỚC": "TỔNG SỐ",
    "Khấu hao nhà ở dân cư": "Khấu hao nhà ở dân cư",
}

for nganh in dfs_gdp_ss["Ngành"].unique().tolist():
    assert nganh in convert_nganh.keys(), nganh

dfs_gdp_ss["Ngành"] = dfs_gdp_ss["Ngành"].map(convert_nganh)
dfs_gdp_ss

,Ngành,Tổng số,YOY,Date,Percent
0,TỔNG SỐ,1.396660e+06,5.657544,2024-03-29,1.000000
1,"Nông, lâm nghiệp và thủy sản",1.456001e+05,2.979307,2024-03-29,0.104249
2,Nông nghiệp,1.122149e+05,2.805032,2024-03-29,0.080345
3,Lâm nghiệp,5.908701e+03,4.083476,2024-03-29,0.004231
4,Thủy sản,2.747649e+04,3.459557,2024-03-29,0.019673
...,...,...,...,...,...
1141,Giáo dục và đào tạo,1.430664e+04,7.82,2014-03-28,0.028404
1142,Y tế và hoạt động trợ giúp xã hội,6.434305e+03,7.80182,2014-03-28,0.012775
1143,"Nghệ thuật, vui chơi và giải trí",3.831353e+03,9.12,2014-03-28,0.007607
1144,Hoạt động dịch vụ khác,9.223516e+03,7.44,2014-03-28,0.018312


In [26]:
dfs_gdp_ss.to_csv("../csv/gdp_ss.csv")

In [27]:
# nganh = "TỔNG SỐ"
# dfs_gdp_ss_per_nganh = dfs_gdp_ss[dfs_gdp_ss["Ngành"] == nganh]

# dfs_gdp_ss_per_nganh["PCT_YOY"] = dfs_gdp_ss_per_nganh["Tổng số"].pct_change(-4)
# print(dfs_gdp_ss_per_nganh)

In [28]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_gdp_ss[
    (
        (dfs_gdp_ss["Ngành"] == "TỔNG SỐ")
        | (dfs_gdp_ss["Ngành"] == "Nông, lâm nghiệp và thủy sản")
        | (dfs_gdp_ss["Ngành"] == "Công nghiệp và xây dựng")
        | (dfs_gdp_ss["Ngành"] == "Dịch vụ")
    )
]

fig = px.area(df_show, x="Date", y="YOY", color="Ngành")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()

In [29]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

df_show = dfs_gdp_ss[
    (
        (dfs_gdp_ss["Ngành"] == "Công nghiệp chế biến, chế tạo")
        | (
            dfs_gdp_ss["Ngành"]
            == "Sản xuất và phân phối điện, khí đốt, nước nóng, hơi nước và điều hòa không khí"
        )
    )
]

fig = px.area(df_show, x="Date", y="YOY", color="Ngành")

fig.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=[
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(count=3, label="3y", step="year", stepmode="backward"),
            dict(count=5, label="5y", step="year", stepmode="backward"),
            dict(count=7, label="7y", step="year", stepmode="backward"),
            dict(count=10, label="10y", step="year", stepmode="backward"),
            dict(step="all"),
        ]
    ),
)

fig.show()